In [15]:
import pandas as pd
from datasets import load_from_disk, Dataset

import torch
from transformers import (
    AutoModelForMaskedLM, DataCollatorForLanguageModeling,
    AutoTokenizer,
    TrainingArguments, Trainer
)

In [3]:
validation_dataset = load_from_disk("/opt/ml/data/train_dataset/validation")
test_dataset = load_from_disk("/opt/ml/data/test_dataset/validation")

In [8]:
val_df = Dataset.to_pandas(validation_dataset)
test_df = Dataset.to_pandas(test_dataset)

In [17]:
pd.DataFrame(val_df.loc[:, 'question'])

,question
0,처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?
1,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?
2,촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?
3,로타이르가 백조를 구하기 위해 사용한 것은?
4,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?
...,...
235,전단이 연나라와의 전쟁에서 승리했을 당시 제나라의 왕은 누구인가?
236,공놀이 경기장 중 일부는 어디에 위치하고 있나?
237,창씨개명령의 시행일을 미루는 것을 수락한 인물은?
238,망코 잉카가 쿠스코를 되찾기 위해 마련한 군사는 총 몇 명인가?


In [22]:
mlm_df = pd.concat([pd.DataFrame(val_df.loc[:, 'question']), pd.DataFrame(test_df.loc[:, 'question'])], ignore_index=True)

In [23]:
mlm_df

,question
0,처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?
1,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?
2,촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?
3,로타이르가 백조를 구하기 위해 사용한 것은?
4,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?
...,...
835,타입 2 가이아 메모리을 만든 집단은?
836,장면이 정치보복에 반대하는 입장에서 처벌을 원치 않은 대상은?
837,"콜드게임 중 어떠한 계기로 인해 잠시 중단된 뒤, 익일에 게임이 진행되는 것은?"
838,제2캐나다기갑여단이 상륙한 곳은?


In [24]:
mlm_dataset = Dataset.from_pandas(mlm_df)
mlm_dataset

Dataset({
    features: ['question'],
    num_rows: 840
})

In [5]:
model_name = 'klue/roberta-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

In [27]:
text_column_name = 'question'
def tokenize_func(examples):
    return tokenizer(examples[text_column_name],
                     truncation=True,
                     return_special_tokens_mask=False,
                     return_token_type_ids=False)

In [28]:
column_names = mlm_dataset.column_names
tokenized_datasets = mlm_dataset.map(
                tokenize_func,
                batched=True,
                remove_columns=column_names
                )

  0%|          | 0/1 [00:00<?, ?ba/s]

In [29]:
tokenized_datasets

Dataset({
    features: ['attention_mask', 'input_ids'],
    num_rows: 840
})

In [31]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./models/roberta_mlm",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=8,
    save_steps=1000,
    save_total_limit=2,
    prediction_loss_only=True,
    report_to = None,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
torch.cuda.is_available()

True

In [33]:
trainer.train()
trainer.save_model("./models/roberta_mlm")

Step,Training Loss
